In [3]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [4]:
print(tf.__version__)

2.5.0


In [5]:
import os
import time,io,shutil,PIL,datetime
from PIL import Image
#import numpy as np
import matplotlib.pyplot as plt
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"

#import tensorflow_addons as tfa
from tensorflow.keras import layers, models
from IPython import display
#tf.config.set_soft_device_placement(True)
#tf.debugging.set_log_device_placement(True)

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.98)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [7]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(0.0001,.9)

In [9]:
# Chnage the working directory on which i have stored the images
# os.chdir("D:")

In [10]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_png(image)

    image = tf.cast(image, tf.float32)
    return image

In [11]:
def resize(input_image, height=286, width=286):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image

In [ ]:
def random_crop1(input_image):
    stacked_image = tf.stack(input_image, axis=0)
    cropped_image = tf.image.random_crop(
      stacked_image, size=[ 256, 256, 3],seed=30)

    return cropped_image

In [ ]:
# normalizing the images to [0, 1]

def normalize(input_image):
    input_image = (input_image / 255)
    return input_image

In [ ]:
#@tf.function
def random_jitter1(input_image):
    # resizing to 286 x 286 x 3
    input_image = resize(input_image, 512,512)
    input_image = random_crop1(input_image)
    
    return input_image
    

In [ ]:
def load_train(path):
    im = load(path)
    im = random_jitter1(im)
    im = normalize(im)
    return im

                             Define Input Pipeline

In [ ]:
batch_size = 1

In [ ]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_blur0 = tf.data.Dataset.list_files("D:/train_blur/*/*.png",seed=4)
train_blur = train_blur0.map(load_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
train_sharp = tf.data.Dataset.list_files("D:/train_sharp/*/*.png",seed=4)
train_sharp = train_sharp.map(load_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
train_dataset = tf.data.Dataset.zip((train_blur,train_sharp)).shuffle(500).batch(batch_size).repeat()
train_dataset

In [ ]:
val_blur0 = tf.data.Dataset.list_files("D:/val_blur/*/*.png",seed=5)
val_blur = val_blur0.map(load_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
val_sharp = tf.data.Dataset.list_files("D:/val_sharp/*/*.png",seed=5)
val_sharp = val_sharp.map(load_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
val_dataset = tf.data.Dataset.zip((val_blur,val_sharp)).batch(batch_size)
val_dataset

In [ ]:
test_blur0 = tf.data.Dataset.list_files("D:/test_blur/*/*.png",seed=41)
test_blur = test_blur0.map(load_train)

In [ ]:
test_sharp = tf.data.Dataset.list_files("D:/test_sharp/*/*.png",seed=41)
test_sharp = test_sharp.map(load_train)

In [ ]:
test_dataset = tf.data.Dataset.zip((test_blur,test_sharp)).batch(batch_size)
test_dataset

In [ ]:
# plot a sample of blurred image
for t in train_blur.take(1):
    plt.imshow(t[:,:,0],cmap="gray")
    plt.axis("off")
    #plt.title("shape {}".format(t.shape))
plt.show()

In [ ]:
# Plot a sample of sharp image
for t in train_sharp.take(1):
    plt.imshow(t[:,:,0],cmap="gray")
    plt.axis("off")
    #plt.title("shape {}".format(t.shape))
plt.show()

In [ ]:
def block(x1,f,size,name = None):
    
    inp = x1
    
    x = layers.Conv2D(f,size,strides=1,padding="same")(x1)
    x = layers.LeakyReLU(0.2)(x)
    x2 = layers.Concatenate()([x,inp])
    
    
    
    x3 = layers.Conv2D(f,size,strides=1,padding="same")(x2)
    x3 = layers.LeakyReLU(0.2)(x3)
    x3 = layers.Concatenate()([x3,x2,inp])
    #
    
    
    x4 = layers.Conv2D(f,size,strides=1,padding="same")(x3)
    x4 = layers.LeakyReLU(0.2)(x4)
    x4 = layers.Concatenate()([x4,x3,x2,inp])
    #
    
    x5 = layers.Conv2D(f,size,strides=1,padding="same")(x4)
    x5 = layers.LeakyReLU(0.2)(x5)
    x5 = layers.Concatenate()([x5,x4,x3,x2,inp])
    
    # final block
    x7 = layers.Conv2D(f,1,strides=1,padding="same",name="block{}_conv".format(name))(x5)
    # residlual learning RL
    
    RL = layers.Add()([x7,inp])
    
    return RL

In [ ]:
def block_s(x1,f,size,name = None):
    
    inp = x1
    
    x = layers.Conv2D(f,(size,size),strides=1,padding="same")(x1)
    x = layers.LeakyReLU(0.2)(x)
    x2 = layers.Concatenate()([x,inp])
    
    
    x3 = layers.Conv2D(f,1,padding="same")(x2)
    x3 = layers.Conv2D(f,(size,size),strides=1,padding="same")(x3)
    x3 = layers.LeakyReLU(0.2)(x3)
    x3 = layers.Concatenate()([x3,x2,inp])
    #
    x4 = layers.Conv2D(f,(1,1),strides=1,padding="same")(x3)
    
    x4 = layers.Conv2D(f,(size,size),strides=1,padding="same")(x4)
    x4 = layers.LeakyReLU(0.2)(x4)
    x4 = layers.Concatenate()([x4,x3,x2,inp])
    #
    x5 = layers.Conv2D(f,(1,1),strides=1,padding="same")(x4)
    
    x5 = layers.Conv2D(f,(size,size),strides=1,padding="same")(x5)
    x5 = layers.LeakyReLU(0.2)(x5)
    x5 = layers.Concatenate()([x5,x4,x3,x2,inp])
    
    # final block
    x7 = layers.Conv2D(f,(1,1),strides=1,padding="same")(x5)
    # residlual learning RL
    
    RL = layers.Add()([x7,inp])
    
    return RL

In [ ]:
def block_c(x1,f,size,name = None):
    
    inp0=x1
    x1 = layers.MaxPooling2D()(x1)
    inp = x1
    x = layers.Conv2D(f,size,strides=1,padding="same",use_bias = False)(x1)
    x = layers.LeakyReLU(0.2)(x)
    x2 = layers.Concatenate()([x,inp,x1])
    
    
    x33 = layers.Conv2D(f,1,padding="same",use_bias = False)(x2)
    x3 = layers.Conv2D(f,size,strides=1,padding="same",use_bias = False)(x33)
    x3 = layers.LeakyReLU(0.2)(x3)
    x3 = layers.Concatenate()([x33,x3,x2,inp])
    #
    x44 = layers.Conv2D(f,1,padding="same",use_bias = False)(x3)
    x4 = layers.Conv2D(f,size,strides=1,padding="same",use_bias = False)(x44)
    x4 = layers.LeakyReLU(0.2)(x4)
    x4 = layers.Concatenate()([x44,x4,x3,x2,inp])
    #
    x55 = layers.Conv2D(f,1,padding="same",use_bias = False)(x4)
    x5 = layers.Conv2D(f,size,strides=1,padding="same",use_bias = False)(x55)
    x5 = layers.LeakyReLU(0.2)(x5)
    x5 = layers.Concatenate()([x55,x5,x4,x3,x2,inp])
    
    # final block
    x5 = tf.nn.depth_to_space(x5,2)
    x7 = layers.Conv2D(f,1,strides=1,padding="same",name="block{}_conv".format(name))(x5)
    # residlual learning RL
    
    RL = layers.Add()([x7,inp0])
    
    return RL

In [ ]:
def block_g(x1,f,size,name = None):
    
    inp = x1
    
    x = layers.Conv2D(f,size,strides=1,padding="same")(x1)
    x = layers.LeakyReLU(0.2)(x)
    x2 = layers.Concatenate()([x,inp])
    
    
    x3 = tf.nn.depth_to_space(x2,2)
    x3 = layers.Conv2D(f,size,strides=2,padding="same",use_bias = False)(x3)
    x3 = layers.LeakyReLU(0.2)(x3)
    x3 = layers.Concatenate()([x3,x2,inp])
    #
    x4 = tf.nn.depth_to_space(x3,2)
    x4 = layers.Conv2D(f,size,strides=2,padding="same",use_bias = False)(x4)
    x4 = layers.LeakyReLU(0.2)(x4)
    x4 = layers.Concatenate()([x4,x3,x2,inp])
    #
    x5 = tf.nn.depth_to_space(x4,2)
    x5 = layers.Conv2D(f,size,strides=2,padding="same",use_bias = False)(x5)
    x5 = layers.LeakyReLU(0.2)(x5)
    x5 = layers.Concatenate()([x5,x4,x3,x2,inp])
    
    # final block
    x7 = layers.Conv2D(f,1,strides=1,padding="same",name="block{}_conv".format(name))(x5)
    # residlual learning RL
    
    RL = layers.Add()([x7,inp])
    
    return RL

In [ ]:
#from tensorflow.keras.layers import Lambda

def Residual_Net(input_shape=(256,256,3)):
    
    input0 = layers.Input(shape=input_shape)
    
    
    
    # shallow features extraction
    sf1 = layers.Conv2D(64,7,strides=2,padding="same",use_bias = False)(input0) 
    
    sf2 = layers.Conv2D(64,3,strides=1,padding="same",use_bias = False)(sf1) 
    # Blocks
    B1 = block_s(sf2,64,3,1)
    B2 = block_s(B1,64,3,2)
    B3 = block_s(B2,64,3,3)
    B4 = block_c(B3,64,3,4)
    #B5 = block(B4,64,3)
    
    # Global features fusion
    
    GFF = layers.Concatenate()([B1,B2,B3,B4])
    GFF = layers.Conv2D(64,1,strides=1,padding="same",use_bias = False)(GFF) 
    GFF = layers.Conv2D(64,3,strides=1,padding="same",use_bias = False)(GFF) # 
    
    # Global Residual Learning GRL
    GRL = layers.Add()([GFF,sf1]) # 
    
    # ESPCNN
    
    x = layers.Conv2D(64,5,activation="relu",padding="same")(GRL)
    x = layers.Conv2D(64,3,activation="relu",padding="same",use_bias = False)(x)
    x = layers.Conv2D(64,3,activation="relu",padding="same",use_bias = False)(x)
    x = tf.nn.depth_to_space(x,4)
    
    
    
    x = layers.Conv2D(3,7,padding="same",strides=2,use_bias = False)(x)
    
    #
    
    
    x = layers.Add()([x,input0])
    x = layers.Activation("tanh")(x)
    
    

    model = models.Model(input0,x)
    model.summary()
    return model

In [ ]:
generator = Residual_Net()
#model = efficient1()

In [ ]:
def generate_images(model, test_input, tar,tr=True):
    prediction = model(test_input, training=tr)
    fig = plt.figure(figsize=(15,15))

    display_list = [test_input[0], tar[0], prediction[0]]
    p = tf.convert_to_tensor(prediction)
    ssim = tf.constant(tf.image.ssim(p,tar,1.0)).numpy()
    psnr = tf.constant(tf.image.psnr(p,tar,1.0)).numpy()
    #noisy_ssim = tf.constant(tf.image.ssim(test_input,tar,1.0)).numpy()
    #noisy_psnr = tf.constant(tf.image.psnr(test_input,tar,1.0)).numpy()
    title = ['Input Image ', 
             'Ground Truth', 
             'Predicted Image  '+str("PSNR/SSIM:{:1.4f}/{:3.4f}".format(list(psnr)[0],list(ssim)[0]))]
    for i in range(3):
        plt.subplot(3, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.axis('off')
        plt.imshow(display_list[i]*0.5+0.5)
        plt.grid(True)
        
    return fig

In [ ]:
for test,tar in test_dataset.take(10):
    generate_images(generator,test,tar,tr=False)


In [ ]:
import functools

psnr_metric = functools.partial(tf.image.psnr, max_val=1.)
psnr_metric.__name__ = 'psnr'

In [ ]:
ssim_metric = functools.partial(tf.image.ssim,max_val=1.)
ssim_metric.__name__ = "ssim"

In [ ]:
num_epochs = 30

In [ ]:
from tensorflow.keras.applications import VGG19
def vgg_layers(layer_names):
    vgg = VGG19(include_top=False,weights="imagenet",input_shape=(256,256,3))
    vgg.trainable = False
    out = [vgg.get_layer(name).output for name in layer_names]
    model = models.Model([vgg.input],out)
    return model

In [ ]:
layer_name = ["block3_conv3"]
feature_extractor = vgg_layers(layer_name)

In [ ]:
def perceptual_loss(y_true,y_pred):
    f_true = feature_extractor(y_true)
    f_pred = feature_extractor(y_pred)
    return 0.80*tf.reduce_mean(tf.abs(f_true-f_pred))+0.20*tf.reduce_mean(tf.abs(y_true-y_pred))

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
generator.compile(loss= perceptual_loss,optimizer=optimizer,metrics=[psnr_metric,ssim_metric])


In [ ]:
# set up checkpoint
checkpoint_path = "training_REDS\cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# create a callbacks for saving weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                save_weights_only=True,
                                                verbose=1)

In [ ]:
shutil.rmtree("story",ignore_errors=True)
log_dir = os.path.join("story"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#logdir = os.path.join("log","scalar")
callbacks  = [# Callback to interrupt the training if the validation loss/metrics stops improving for some epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss',
                                     restore_best_weights=True),tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1),cp_callback]

In [ ]:
start = time.time()
history = generator.fit(
    train_dataset, steps_per_epoch=21000, epochs=num_epochs,
    validation_data=val_dataset,validation_steps=3000,
    verbose=1,callbacks=callbacks)
print(f"Total number of Days taken for running the model: {(time.time()-start)/(3600*24)} Days")

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(10, 5), sharex='col')
ax[0, 0].set_title("loss")
ax[0, 1].set_title("val-loss")
ax[1, 0].set_title("psnr")
ax[1, 1].set_title("val-psnr")
ax[2, 0].set_title("ssim")
ax[2, 1].set_title("val-ssim")


ax[0, 0].plot(history.history['loss'])
ax[0, 1].plot(history.history['val_loss'])
ax[1, 0].plot(history.history['psnr'])
ax[1, 1].plot(history.history['val_psnr'])
ax[2, 0].plot(history.history['ssim'])
ax[2, 1].plot(history.history['val_ssim'])

fig.savefig("motion_blur.png")

In [ ]:
for noisy,target in test_dataset.take(10):
    generate_images(generator,noisy,target)

In [ ]:
f = []
for n,t in test_dataset.take(200):
    f.append(generate_images(generator,n,t))

In [ ]:
def fig2img(fig):
    buf = io.BytesIO()
    fig.savefig(buf)
    plt.close()
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
frame = []
for i in f:
    frame.append(fig2img(i))

In [ ]:
frame[0].save("test0.png")
frame[1].save("test1.png")
frame[2].save("test2.png")

In [ ]:
frame[0].save("Result_REDS.gif",format="GIF",append_images=frame[1:],save_all=True,duration=2000,loop=0)

In [ ]:
ls {checkpoint_dir}

In [ ]:
generator.load_weights(checkpoint_dir)

In [ ]:
m = generator.evaluate(test_dataset.take(20))

In [ ]:
m